In [14]:
from neo4j import GraphDatabase

# ----------------------------------------------------------------------
# Configuration
# ----------------------------------------------------------------------
# Bolt protocol URI
# Neo4j uses port 7687 for Bolt connections by default.
# Port 7474 is for the HTTP API and the Neo4j Browser.
URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "nico2002" # IMPORTANT: Replace with your actual password!
                                  # If you haven't changed it, it's 'neo4j' initially.


In [16]:
# ----------------------------------------------------------------------
# Connection and Interaction
# ----------------------------------------------------------------------

class Neo4jConnector:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        print(f"Attempting to connect to Neo4j at {uri}...")

        # Optional: Verify connection immediately
        try:
            self.driver.verify_connectivity()
            print("Neo4j driver connectivity verified.")
        except Exception as e:
            print(f"Failed to connect to Neo4j at {uri}: {e}")
            print("Please ensure Neo4j is running and accessible.")
            raise # Re-raise to stop execution if connection fails

    def close(self):
        self.driver.close()
        print("Neo4j connection closed.")

    def run_query(self, query, parameters=None, fetch_single_record=False):
        """
        Runs a Cypher query and returns the results.
        'parameters' should be a dictionary for parameterized queries.
        """
        if parameters is None:
            parameters = {}
        with self.driver.session() as session:
            try:
                result = session.run(query, parameters)
                print(f"Query executed: '{query}' with parameters {parameters}")
                
                if fetch_single_record:
                    return result.single()
                else:
                    return list(result)
            except Exception as e:
                print(f"Error executing query: {e}")
                raise # Re-raise the exception after printing

    def create_person(self, name, age):
        """Example: Creates a new Person node."""
        query = (
            "CREATE (p:Person {name: $name, age: $age})"
            "RETURN p.name AS name, p.age AS age"
        )
        return self.run_query(query, {"name": name, "age": age})

    def find_person(self, name):
        """Example: Finds a Person node by name."""
        query = (
            "MATCH (p:Person {name: $name})"
            "RETURN p.name AS name, p.age AS age"
        )
        return self.run_query(query, {"name": name})

    def get_node_count(self):
        """Example: Gets the total number of nodes in the database."""
        query = "MATCH (n) RETURN count(n) AS node_count"
        result = self.run_query(query)  
        return result[0]["node_count"]


In [17]:

QUERY_FOOD = """LOAD CSV WITH HEADERS FROM 'file:///Food.csv' as row
MERGE (food: Food {id_food:row.id_food})
    ON CREATE SET food.code = row.code, food.name = row.name""" 

QUERY_ANIMALS = """LOAD CSV WITH HEADERS FROM 'file:///Animals.csv' as row
MERGE (animal: Animal {id_animals:row.id_animals})
    ON CREATE SET animal.code = row.code, animal.name = row.name""" 

QUERY_FERTILIZER = """LOAD CSV WITH HEADERS FROM 'file:///Fertilizer.csv' as row
MERGE (fertilizer: Fertilizer {id_fertilizer:row.id_fertilizer})
    ON CREATE SET fertilizer.code = row.code, fertilizer.name = row.name""" 

QUERY_RESIDUE = """LOAD CSV WITH HEADERS FROM 'file:///Residue.csv' as row
MERGE (residue: Residue {id_residue:row.id_residue})
    ON CREATE SET residue.code = row.code, residue.name = row.name""" 

QUERY_GASES = """LOAD CSV WITH HEADERS FROM 'file:///Gases.csv' as row
MERGE (gases: Gases {id_gases:row.id_gases})
    ON CREATE SET gases.code = row.code, gases.name = row.name""" 

QUERY_AREA = """LOAD CSV WITH HEADERS FROM 'file:///Area.csv' as row
MERGE (area: Area {id_area:row.id_area})
    ON CREATE SET area.code = row.code, area.country = row.country""" 




CREATE_INDEX_FOOD = """CREATE INDEX foodID FOR (food:Food) ON (food.id_food);"""

CREATE_INDEX_ANIMALS = """CREATE INDEX animalsID FOR (animals:Animals) ON (animals.id_animals);"""

CREATE_INDEX_FERTILIZER = """CREATE INDEX fertilizerID FOR (fertilizer:Fertilizer) ON (fertilizer.id_fertilizer);"""

CREATE_INDEX_RESIDUE = """CREATE INDEX residueID FOR (residue:Residue) ON (residue.id_residue)"""

# CREATE_INDEX_GASES = """CREATE INDEX gasesID FOR (gases:Gases) ON (gases.id_gases);"""

CREATE_INDEX_AREA = """CREATE INDEX areaID FOR (area:Area) ON (area.id_area)"""



connector = Neo4jConnector(URI, USERNAME, PASSWORD)

try:
    
    connector.run_query(QUERY_FOOD)
    connector.run_query(QUERY_FERTILIZER)
    connector.run_query(QUERY_RESIDUE)
    connector.run_query(QUERY_ANIMALS)
    connector.run_query(QUERY_AREA)

    # ONLY ONCE
    connector.run_query(CREATE_INDEX_FOOD)
    connector.run_query(CREATE_INDEX_ANIMALS)
    connector.run_query(CREATE_INDEX_FERTILIZER)
    connector.run_query(CREATE_INDEX_RESIDUE)
    connector.run_query(CREATE_INDEX_AREA)
    connector.run_query("CALL db.awaitIndexes();")



    # connector.run_query(QUERY_GASES)
    # connector.run_query(CREATE_INDEX_GASES)



except Exception as e:
    print(f"An error occurred during script execution: {e}")
finally:
    connector.close()



Attempting to connect to Neo4j at bolt://localhost:7687...
Neo4j driver connectivity verified.
Query executed: 'LOAD CSV WITH HEADERS FROM 'file:///Food.csv' as row
MERGE (food: Food {id_food:row.id_food})
    ON CREATE SET food.code = row.code, food.name = row.name' with parameters {}
Query executed: 'LOAD CSV WITH HEADERS FROM 'file:///Fertilizer.csv' as row
MERGE (fertilizer: Fertilizer {id_fertilizer:row.id_fertilizer})
    ON CREATE SET fertilizer.code = row.code, fertilizer.name = row.name' with parameters {}
Query executed: 'LOAD CSV WITH HEADERS FROM 'file:///Residue.csv' as row
MERGE (residue: Residue {id_residue:row.id_residue})
    ON CREATE SET residue.code = row.code, residue.name = row.name' with parameters {}
Query executed: 'LOAD CSV WITH HEADERS FROM 'file:///Animals.csv' as row
MERGE (animal: Animal {id_animals:row.id_animals})
    ON CREATE SET animal.code = row.code, animal.name = row.name' with parameters {}
Query executed: 'LOAD CSV WITH HEADERS FROM 'file:///Area

In [20]:

QUERY_LOAD_STOCKS_RELATION = """LOAD CSV WITH HEADERS FROM 'file:///Stocks.csv' as row
MATCH (animal:Animal {id_animals: row.id_animals})
MATCH (area:Area {id_area: row.id_area})
MERGE (animal) - [s:STOCKS {year: toInteger(row.year)}] - (area)
    ON CREATE SET s.year = toInteger(row.year), s.unit = row.unit, s.value = toFloat(row.value);
"""


QUERY_LOAD_PRODUCTION_RELATION  = """LOAD CSV WITH HEADERS FROM 'file:///Production.csv' as row
CALL {
    WITH row
    MATCH (food:Food {id_food: row.id_food})
    MATCH (area:Area {id_area: row.id_area})
    MERGE (food)-[s:PRODUCTION {year: toInteger(row.year)}]-(area)
    ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)
} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed
"""

QUERY_LOAD_AREA_HARVEST_RELATION  = """LOAD CSV WITH HEADERS FROM 'file:///Area_harvest.csv' as row
CALL {
    WITH row
    MATCH (food:Food {id_food: row.id_food})
    MATCH (area:Area {id_area: row.id_area})
    MERGE (food)-[s:AREA_HARVEST {year: toInteger(row.year)}]-(area)
    ON CREATE SET s.scale = row.scale, s.value = toFloat(row.value)
} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed
"""


QUERY_LOAD_USE_PER_AREA_RELATION  = """LOAD CSV WITH HEADERS FROM 'file:///Use_per_area.csv' as row
CALL {
    WITH row
    MATCH (fertilizer:Fertilizer {id_fertilizer: row.id_fertilizer})
    MATCH (area:Area {id_area: row.id_area})
    MERGE (fertilizer)-[s:USE_PER_AREA {year: toInteger(row.year)}]-(area)
    ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)
} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed
"""


QUERY_LOAD_AGRICULTURE_USE_RELATION  = """LOAD CSV WITH HEADERS FROM 'file:///Agriculture_use.csv' as row
CALL {
    WITH row
    MATCH (fertilizer:Fertilizer {id_fertilizer: row.id_fertilizer})
    MATCH (area:Area {id_area: row.id_area})
    MERGE (fertilizer)-[s:AGRICULTURE_USE {year: toInteger(row.year)}]-(area)
    ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)
} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed
"""

QUERY_LOAD_FERTILIZER_PRODUCTION_RELATION  = """LOAD CSV WITH HEADERS FROM 'file:///Fertilizer_production.csv' as row
CALL {
    WITH row
    MATCH (fertilizer:Fertilizer {id_fertilizer: row.id_fertilizer})
    MATCH (area:Area {id_area: row.id_area})
    MERGE (fertilizer)-[s:FERTILIZER_PRODUCTION {year: toInteger(row.year)}]-(area)
    ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)
} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed
"""

QUERY_LOAD_EMISSION_RELATION  = """LOAD CSV WITH HEADERS FROM 'file:///newEmission.csv' as row
CALL {
    WITH row
    MATCH (residue:Residue {id_residue: row.id_residue})
    MATCH (area: Area {id_area: row.id_area})
    MERGE (area)-[s:EMISSIONS {year: toInteger(row.year),gases:row.name}]-(residue)
        ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)
} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed
"""


connector = Neo4jConnector(URI, USERNAME, PASSWORD)
try:
    
    connector.run_query(QUERY_LOAD_STOCKS_RELATION)
    connector.run_query(QUERY_LOAD_PRODUCTION_RELATION)
    connector.run_query(QUERY_LOAD_AREA_HARVEST_RELATION)
    connector.run_query(QUERY_LOAD_AGRICULTURE_USE_RELATION)
    connector.run_query(QUERY_LOAD_USE_PER_AREA_RELATION)
    connector.run_query(QUERY_LOAD_FERTILIZER_PRODUCTION_RELATION)
    # connector.run_query(QUERY_LOAD_EMISSION_RELATION)




except Exception as e:
    print(f"An error occurred during script execution: {e}")
finally:
    connector.close()


Attempting to connect to Neo4j at bolt://localhost:7687...
Neo4j driver connectivity verified.
Query executed: 'LOAD CSV WITH HEADERS FROM 'file:///Stocks.csv' as row
MATCH (animal:Animal {id_animals: row.id_animals})
MATCH (area:Area {id_area: row.id_area})
MERGE (animal) - [s:STOCKS {year: toInteger(row.year)}] - (area)
    ON CREATE SET s.year = toInteger(row.year), s.unit = row.unit, s.value = toFloat(row.value);
' with parameters {}


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (row) { ... }} {position: line: 2, column: 1, offset: 59} for query: "LOAD CSV WITH HEADERS FROM 'file:///Production.csv' as row\nCALL {\n    WITH row\n    MATCH (food:Food {id_food: row.id_food})\n    MATCH (area:Area {id_area: row.id_area})\n    MERGE (food)-[s:PRODUCTION {year: toInteger(row.year)}]-(area)\n    ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)\n} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed\n"


Query executed: 'LOAD CSV WITH HEADERS FROM 'file:///Production.csv' as row
CALL {
    WITH row
    MATCH (food:Food {id_food: row.id_food})
    MATCH (area:Area {id_area: row.id_area})
    MERGE (food)-[s:PRODUCTION {year: toInteger(row.year)}]-(area)
    ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)
} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed
' with parameters {}


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (row) { ... }} {position: line: 2, column: 1, offset: 61} for query: "LOAD CSV WITH HEADERS FROM 'file:///Area_harvest.csv' as row\nCALL {\n    WITH row\n    MATCH (food:Food {id_food: row.id_food})\n    MATCH (area:Area {id_area: row.id_area})\n    MERGE (food)-[s:AREA_HARVEST {year: toInteger(row.year)}]-(area)\n    ON CREATE SET s.scale = row.scale, s.value = toFloat(row.value)\n} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed\n"


Query executed: 'LOAD CSV WITH HEADERS FROM 'file:///Area_harvest.csv' as row
CALL {
    WITH row
    MATCH (food:Food {id_food: row.id_food})
    MATCH (area:Area {id_area: row.id_area})
    MERGE (food)-[s:AREA_HARVEST {year: toInteger(row.year)}]-(area)
    ON CREATE SET s.scale = row.scale, s.value = toFloat(row.value)
} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed
' with parameters {}


...

Query executed: 'LOAD CSV WITH HEADERS FROM 'file:///Agriculture_use.csv' as row
CALL {
    WITH row
    MATCH (fertilizer:Fertilizer {id_fertilizer: row.id_fertilizer})
    MATCH (area:Area {id_area: row.id_area})
    MERGE (fertilizer)-[s:AGRICULTURE_USE {year: toInteger(row.year)}]-(area)
    ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)
} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed
' with parameters {}


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (row) { ... }} {position: line: 2, column: 1, offset: 61} for query: "LOAD CSV WITH HEADERS FROM 'file:///Use_per_area.csv' as row\nCALL {\n    WITH row\n    MATCH (fertilizer:Fertilizer {id_fertilizer: row.id_fertilizer})\n    MATCH (area:Area {id_area: row.id_area})\n    MERGE (fertilizer)-[s:USE_PER_AREA {year: toInteger(row.year)}]-(area)\n    ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)\n} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed\n"


Query executed: 'LOAD CSV WITH HEADERS FROM 'file:///Use_per_area.csv' as row
CALL {
    WITH row
    MATCH (fertilizer:Fertilizer {id_fertilizer: row.id_fertilizer})
    MATCH (area:Area {id_area: row.id_area})
    MERGE (fertilizer)-[s:USE_PER_AREA {year: toInteger(row.year)}]-(area)
    ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)
} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed
' with parameters {}


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (row) { ... }} {position: line: 2, column: 1, offset: 70} for query: "LOAD CSV WITH HEADERS FROM 'file:///Fertilizer_production.csv' as row\nCALL {\n    WITH row\n    MATCH (fertilizer:Fertilizer {id_fertilizer: row.id_fertilizer})\n    MATCH (area:Area {id_area: row.id_area})\n    MERGE (fertilizer)-[s:FERTILIZER_PRODUCTION {year: toInteger(row.year)}]-(area)\n    ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)\n} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed\n"


Query executed: 'LOAD CSV WITH HEADERS FROM 'file:///Fertilizer_production.csv' as row
CALL {
    WITH row
    MATCH (fertilizer:Fertilizer {id_fertilizer: row.id_fertilizer})
    MATCH (area:Area {id_area: row.id_area})
    MERGE (fertilizer)-[s:FERTILIZER_PRODUCTION {year: toInteger(row.year)}]-(area)
    ON CREATE SET s.unit = row.unit, s.value = toFloat(row.value)
} IN TRANSACTIONS OF 100000 ROWS; // Adjust the row count as needed
' with parameters {}
Neo4j connection closed.
